In [1]:
from features import ensembleFeature
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import torch
from torch import nn
import re
from sklearn.utils import shuffle
from utils import metricsCal
from torch.utils.data import DataLoader,TensorDataset
import math
import sys
import copy
import pickle
from torch.autograd import Variable
from sklearn.model_selection import KFold
import torch.nn.functional as F
import os
import LAModel as Model
from matplotlib import cm
from matplotlib.ticker import LinearLocator
import csv
%matplotlib inline

In [2]:
#一个函数用于序列转换，一个函数用于检测“N”碱基
def long_short(data):
    seq_list = []
    for i in data[0]:
        seq_list.append(str(i)[480:521])
    return np.array(seq_list)

def check_N(data1,data2):
    seq_list = []
    gene_list = []
    for i in range(len(data1)):
        if str(data1[i]).find("N")<0:
            seq_list.append(data1[i])
            gene_list.append(data2[i])
    return np.array(seq_list),np.array(gene_list)

In [11]:
# type_name = ["FullTranscript","matureRNA"]
# cell_name = ["A549","CD8T","Hek293_abacm","Hek293_sysy","HeLa","MOLM13"]
# feature_name = ["binary","NCPA","gene","emb","PSNP","ENAC","PseDNC","EIIP","PCP","DBPF"]
# network_name = ["Self-Attention+BiLSTM","BILSTM+Self-Attention","BILSTM"]
type_name = ["FullTranscript"]
cell_name = ["A549"]
feature_name = ["NCPA"]
network_name = ["Self-Attention+BiLSTM","BiLSTM+Self-Attention","BiLSTM","Self-Attention","Self-Attention+BiLSTM+No","BiLSTM+Self-Attention+No","Self-Attention+No"]
rows = len(type_name)*len(cell_name)*len(feature_name)*len(network_name)*5
row = -1
max_epochs = 50
max_patience = 10
batch_size = 1024
row_name = []
Loss = np.zeros(rows*max_epochs).reshape(rows,max_epochs)
Acc = np.zeros(rows*(max_epochs+1)).reshape(rows,(max_epochs+1))
Mcc = np.zeros(rows*(max_epochs+1)).reshape(rows,(max_epochs+1))
Auc = np.zeros(rows*(max_epochs+1)).reshape(rows,(max_epochs+1))
th_All = np.zeros(rows*(max_epochs+1)).reshape(rows,(max_epochs+1))

In [12]:
Folder_name = []
for typei in range(len(type_name)):
    for celli in range(len(cell_name)):
        Folder_list = []
        Folder_list.append("data/"+type_name[typei]+"/"+cell_name[celli]+"/Pos_"+cell_name[celli]+"_train_"+type_name[typei]+"_seq.csv")
        Folder_list.append("data/"+type_name[typei]+"/"+cell_name[celli]+"/Neg_"+cell_name[celli]+"_train_"+type_name[typei]+"_seq.csv")
        Folder_list.append("data/"+type_name[typei]+"/"+cell_name[celli]+"/Pos_"+cell_name[celli]+"_test_"+type_name[typei]+"_seq.csv")
        Folder_list.append("data/"+type_name[typei]+"/"+cell_name[celli]+"/Neg_"+cell_name[celli]+"_test_"+type_name[typei]+"_seq.csv")
        Folder_list.append("data/"+type_name[typei]+"/"+cell_name[celli]+"/Pos_"+cell_name[celli]+"_train_"+type_name[typei]+"_gene.csv")
        Folder_list.append("data/"+type_name[typei]+"/"+cell_name[celli]+"/Neg_"+cell_name[celli]+"_train_"+type_name[typei]+"_gene.csv")
        Folder_list.append("data/"+type_name[typei]+"/"+cell_name[celli]+"/Pos_"+cell_name[celli]+"_test_"+type_name[typei]+"_gene.csv")
        Folder_list.append("data/"+type_name[typei]+"/"+cell_name[celli]+"/Neg_"+cell_name[celli]+"_test_"+type_name[typei]+"_gene.csv")
        Folder_name.append(Folder_list)

In [13]:
#这是第一重循环~
#获取8个文件的文件名
for Folderi in range(len(Folder_name)):
    trainPosSeq_file = Folder_name[Folderi][0]
    trainNegSeq_file = Folder_name[Folderi][1]
    testPosSeq_file = Folder_name[Folderi][2]
    testNegSeq_file = Folder_name[Folderi][3]
    trainPosGene_file = Folder_name[Folderi][4]
    trainNegGene_file = Folder_name[Folderi][5]
    testPosGene_file = Folder_name[Folderi][6]
    testNegGene_file = Folder_name[Folderi][7]
    #获取文件信息
    trainPos_seq = pd.read_csv(trainPosSeq_file,header=None)
    trainNeg_seq = pd.read_csv(trainNegSeq_file,header=None)
    testPos_seq = pd.read_csv(testPosSeq_file,header=None)
    testNeg_seq = pd.read_csv(testNegSeq_file,header=None)
    trainPos_gene = pd.read_csv(trainPosGene_file) 
    trainNeg_gene = pd.read_csv(trainNegGene_file)
    testPos_gene = pd.read_csv(testPosGene_file)
    testNeg_gene = pd.read_csv(testNegGene_file)
    #序列转41-nt长度
    trainPos_seq = long_short(trainPos_seq)
    trainNeg_seq = long_short(trainNeg_seq)
    testPos_seq = long_short(testPos_seq)
    testNeg_seq = long_short(testNeg_seq)
    #基因转35长度
    trainPos_gene = trainPos_gene.iloc[:,[0,1,3,4,5,6,7,8,9,10,11,13,14,16,17,19,23,22,25,20,21,44,45,46,47,48,49,50,65,66,69,55,51,57,56]]
    trainNeg_gene = trainNeg_gene.iloc[:,[0,1,3,4,5,6,7,8,9,10,11,13,14,16,17,19,23,22,25,20,21,44,45,46,47,48,49,50,65,66,69,55,51,57,56]]
    testPos_gene = testPos_gene.iloc[:,[0,1,3,4,5,6,7,8,9,10,11,13,14,16,17,19,23,22,25,20,21,44,45,46,47,48,49,50,65,66,69,55,51,57,56]]
    testNeg_gene = testNeg_gene.iloc[:,[0,1,3,4,5,6,7,8,9,10,11,13,14,16,17,19,23,22,25,20,21,44,45,46,47,48,49,50,65,66,69,55,51,57,56]]
    trainPos_seq,trainPos_gene = check_N(trainPos_seq,np.array(trainPos_gene))
    trainNeg_seq,trainNeg_gene = check_N(trainNeg_seq,np.array(trainNeg_gene))
    testPos_seq,testPos_gene = check_N(testPos_seq,np.array(testPos_gene))
    testNeg_seq,testNeg_gene = check_N(testNeg_seq,np.array(testNeg_gene))
    #基因变Float
    trainPos_gene = np.float32(trainPos_gene)
    trainNeg_gene = np.float32(trainNeg_gene)
    testPos_gene = np.float32(testPos_gene)
    testNeg_gene = np.float32(testNeg_gene)
    #打乱顺序，再取等长
    trainNeg_seq, trainNeg_gene = shuffle(trainNeg_seq,trainNeg_gene, random_state=1)
    testNeg_seq, testNeg_gene = shuffle(testNeg_seq, testNeg_gene, random_state=1)

    trainNeg_seq = trainNeg_seq[:len(trainPos_seq)]
    trainNeg_gene = trainNeg_gene[:len(trainPos_gene)]
    testNeg_seq = testNeg_seq[:len(testPos_seq)]
    testNeg_gene = testNeg_gene[:len(testPos_gene)]
    #第二重循环
    for featurei in range(len(feature_name)):
        if feature_name[featurei] == "binary":
            trainPos = ensembleFeature.Binary(trainPos_seq)
            trainNeg = ensembleFeature.Binary(trainNeg_seq)
            testPos = ensembleFeature.Binary(testPos_seq)
            testNeg = ensembleFeature.Binary(testNeg_seq)
        elif feature_name[featurei] == "DNC":
            trainPos = ensembleFeature.DNC(trainPos_seq)
            trainNeg = ensembleFeature.DNC(trainNeg_seq)
            testPos = ensembleFeature.DNC(testPos_seq)
            testNeg = ensembleFeature.DNC(testNeg_seq)
        elif feature_name[featurei] == "NCPA":
            trainPos = ensembleFeature.NCPA(trainPos_seq)
            trainNeg = ensembleFeature.NCPA(trainNeg_seq)
            testPos = ensembleFeature.NCPA(testPos_seq)
            testNeg = ensembleFeature.NCPA(testNeg_seq)
        elif feature_name[featurei] == "gene":
            trainPos = trainPos_gene
            trainNeg = trainNeg_gene
            testPos = testPos_gene
            testNeg = testNeg_gene
        elif feature_name[featurei] == "emb":
            trainPos = ensembleFeature.emb_seqs(trainPos_seq)
            trainNeg = ensembleFeature.emb_seqs(trainNeg_seq)
            testPos = ensembleFeature.emb_seqs(testPos_seq)
            testNeg = ensembleFeature.emb_seqs(testNeg_seq)
        elif feature_name[featurei] == "PSNP":
            trainPos,trainNeg,testPos,testNeg = ensembleFeature.PSNP(trainPos_seq,trainNeg_seq,testPos_seq,testNeg_seq)
        elif feature_name[featurei] == "ENAC":
            trainPos = ensembleFeature.ENAC(trainPos_seq)
            trainNeg = ensembleFeature.ENAC(trainNeg_seq)
            testPos = ensembleFeature.ENAC(testPos_seq)
            testNeg = ensembleFeature.ENAC(testNeg_seq)
        elif feature_name[featurei] == "EIIP":
            trainPos = ensembleFeature.EIIP(trainPos_seq)
            trainNeg = ensembleFeature.EIIP(trainNeg_seq)
            testPos = ensembleFeature.EIIP(testPos_seq)
            testNeg = ensembleFeature.EIIP(testNeg_seq)
        elif feature_name[featurei] == "PseDNC":
            trainPos = ensembleFeature.PseDNC(trainPos_seq)
            trainNeg = ensembleFeature.PseDNC(trainNeg_seq)
            testPos = ensembleFeature.PseDNC(testPos_seq)
            testNeg = ensembleFeature.PseDNC(testNeg_seq)
        elif feature_name[featurei] == "PCP":
            trainPos = ensembleFeature.PCP(trainPos_seq)
            trainNeg = ensembleFeature.PCP(trainNeg_seq)
            testPos = ensembleFeature.PCP(testPos_seq)
            testNeg = ensembleFeature.PCP(testNeg_seq)
        elif feature_name[featurei] == "DBPF":
            trainPos = ensembleFeature.DBPF(trainPos_seq)
            trainNeg = ensembleFeature.DBPF(trainNeg_seq)
            testPos = ensembleFeature.DBPF(testPos_seq)
            testNeg = ensembleFeature.DBPF(testNeg_seq)
        if len(testNeg.shape)==2:
            trainPos = trainPos.reshape(trainPos.shape[0],1,trainPos.shape[1])
            trainNeg = trainNeg.reshape(trainNeg.shape[0],1,trainNeg.shape[1])
            testPos = testPos.reshape(testPos.shape[0],1,testPos.shape[1])
            testNeg = testNeg.reshape(testNeg.shape[0],1,testNeg.shape[1])
            
        trainPos_label = np.ones(len(trainPos))
        trainNeg_label = np.zeros(len(trainNeg))
        testPos_label = np.ones(len(testPos))
        testNeg_label = np.zeros(len(testNeg))

        trainData = np.append(trainPos,trainNeg,axis=0)
        testData = np.append(testPos,testNeg,axis = 0)
        trainLabel = np.append(trainPos_label,trainNeg_label,axis = 0)
        testLabel = np.append(testPos_label,testNeg_label,axis = 0)

        kf = KFold(5,True,10)
        #第三重循环
        for networki in range(len(network_name)):
            if network_name[networki] == "Self-Attention+BiLSTM":
                
                for i,[train_index, test_index] in enumerate(kf.split(trainData)):
                    row+=1
                    row_one = type_name[typei]+"_"+cell_name[celli]+"_"+feature_name[featurei]+"_"+network_name[networki]+"_KFold_" + str(i)
                    row_name.append(row_one)
                    X_train = trainData[train_index]
                    X_test = trainData[test_index]
                    Y_train = trainLabel[train_index]
                    Y_test = trainLabel[test_index]
                    best_auc = 0
                    patience = 0
                    model_dir = "Model_Feature_Extraction/"+type_name[typei]+"_"+cell_name[celli]+"/"+\
                            feature_name[featurei]+"_"+network_name[networki]+"/KFold_" + str(i) + "/"
                    model = Model.ModelSB_Pro(trainData.shape[1],trainData.shape[2])
                    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")  #选择设备
                    os.environ['CUDA_VISIBLE_DEVICES'] = '0'
                    for j in range(max_epochs):
                        runningLoss,th = Model.train(model,X_train,Y_train,i,device,model_dir,batch_size)
                        acc, mcc, auc=Model.test(X_test,Y_test,best_auc,device,model_dir,batch_size,th)
                        Loss[row][j] = runningLoss
                        th_All[row][j] = th
                        Acc[row][j] = acc
                        Mcc[row][j] = mcc
                        Auc[row][j] = auc
                        if auc > best_auc:
                            best_auc=auc
                            patience=0
                        else:
                            patience+=1
                        if patience==max_patience:
                            break
                    tmp_th = 0.5
                    tmp_auc = 0
                    for kk in range(max_epochs):
                        if Auc[row][kk]>tmp_auc:
                            tmp_th = th_All[row][kk]
                            tmp_auc = Auc[row][kk]
                    acc, mcc, auc=Model.independTest(testData,testLabel,device,model_dir,batch_size,tmp_th)
                    Acc[row][-1] = acc
                    Mcc[row][-1] = mcc
                    Auc[row][-1] = auc
                    th_All[row][-1] = tmp_th
            if network_name[networki] == "BiLSTM+Self-Attention":
                for i,[train_index, test_index] in enumerate(kf.split(trainData)):
                    row+=1
                    row_one = type_name[typei]+"_"+cell_name[celli]+"_"+feature_name[featurei]+"_"+network_name[networki]+"_KFold_" + str(i)
                    row_name.append(row_one)
                    X_train = trainData[train_index]
                    X_test = trainData[test_index]
                    Y_train = trainLabel[train_index]
                    Y_test = trainLabel[test_index]
                    best_auc = 0
                    patience = 0
                    model_dir = "Model_Feature_Extraction/"+type_name[typei]+"_"+cell_name[celli]+"/"+\
                            feature_name[featurei]+"_"+network_name[networki]+"/KFold_" + str(i) + "/"
                    model = Model.ModelBS_Pro(trainData.shape[1],trainData.shape[2])
                    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")  #选择设备
                    os.environ['CUDA_VISIBLE_DEVICES'] = '0'
                    for j in range(max_epochs):
                        runningLoss,th = Model.train(model,X_train,Y_train,i,device,model_dir,batch_size)
                        acc, mcc, auc=Model.test(X_test,Y_test,best_auc,device,model_dir,batch_size,th)
                        Loss[row][j] = runningLoss
                        th_All[row][j] = th
                        Acc[row][j] = acc
                        Mcc[row][j] = mcc
                        Auc[row][j] = auc
                        if auc > best_auc:
                            best_auc=auc
                            patience=0
                        else:
                            patience+=1
                        if patience==max_patience:
                            break
                    tmp_th = 0.5
                    tmp_auc = 0
                    for kk in range(max_epochs):
                        if Auc[row][kk]>tmp_auc:
                            tmp_th = th_All[row][kk]
                            tmp_auc = Auc[row][kk]
                    acc, mcc, auc=Model.independTest(testData,testLabel,device,model_dir,batch_size,tmp_th)
                    Acc[row][-1] = acc
                    Mcc[row][-1] = mcc
                    Auc[row][-1] = auc
                    th_All[row][-1] = tmp_th

            if network_name[networki] == "BiLSTM":

                for i,[train_index, test_index] in enumerate(kf.split(trainData)):
                    row+=1
                    row_one = type_name[typei]+"_"+cell_name[celli]+"_"+feature_name[featurei]+"_"+network_name[networki]+"_KFold_" + str(i)
                    row_name.append(row_one)

                    X_train = trainData[train_index]
                    X_test = trainData[test_index]
                    Y_train = trainLabel[train_index]
                    Y_test = trainLabel[test_index]
                    best_auc = 0
                    patience = 0
                    model_dir = "Model_Feature_Extraction/"+type_name[typei]+"_"+cell_name[celli]+"/"+\
                            feature_name[featurei]+"_"+network_name[networki]+"/KFold_" + str(i) + "/"
                    model = Model.ModelB_Pro(trainData.shape[1],trainData.shape[2])
                    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")  #选择设备
                    os.environ['CUDA_VISIBLE_DEVICES'] = '0'
                    for j in range(max_epochs):
                        runningLoss,th = Model.train(model,X_train,Y_train,i,device,model_dir,batch_size)
                        acc, mcc, auc=Model.test(X_test,Y_test,best_auc,device,model_dir,batch_size,th)
                        Loss[row][j] = runningLoss
                        th_All[row][j] = th
                        Acc[row][j] = acc
                        Mcc[row][j] = mcc
                        Auc[row][j] = auc
                        if auc > best_auc:
                            best_auc=auc
                            patience=0
                        else:
                            patience+=1
                        if patience==max_patience:
                             break
                    tmp_th = 0.5
                    tmp_auc = 0
                    for kk in range(max_epochs):
                        if Auc[row][kk]>tmp_auc:
                            tmp_th = th_All[row][kk]
                            tmp_auc = Auc[row][kk]
                    acc, mcc, auc=Model.independTest(testData,testLabel,device,model_dir,batch_size,tmp_th)
                    Acc[row][-1] = acc
                    Mcc[row][-1] = mcc
                    Auc[row][-1] = auc
                    th_All[row][-1] = tmp_th
            if network_name[networki] == "Self-Attention":
                
                for i,[train_index, test_index] in enumerate(kf.split(trainData)):
                    row+=1
                    row_one = type_name[typei]+"_"+cell_name[celli]+"_"+feature_name[featurei]+"_"+network_name[networki]+"_KFold_" + str(i)
                    row_name.append(row_one)
                    X_train = trainData[train_index]
                    X_test = trainData[test_index]
                    Y_train = trainLabel[train_index]
                    Y_test = trainLabel[test_index]
                    best_auc = 0
                    patience = 0
                    model_dir = "Model_Feature_Extraction/"+type_name[typei]+"_"+cell_name[celli]+"/"+\
                            feature_name[featurei]+"_"+network_name[networki]+"/KFold_" + str(i) + "/"
                    model = Model.ModelS_Pro(trainData.shape[1],trainData.shape[2])
                    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")  #选择设备
                    os.environ['CUDA_VISIBLE_DEVICES'] = '0'
                    for j in range(max_epochs):
                        runningLoss,th = Model.train(model,X_train,Y_train,i,device,model_dir,batch_size)
                        acc, mcc, auc=Model.test(X_test,Y_test,best_auc,device,model_dir,batch_size,th)
                        Loss[row][j] = runningLoss
                        th_All[row][j] = th
                        Acc[row][j] = acc
                        Mcc[row][j] = mcc
                        Auc[row][j] = auc
                        if auc > best_auc:
                            best_auc=auc
                            patience=0
                        else:
                            patience+=1
                        if patience==max_patience:
                            break
                    tmp_th = 0.5
                    tmp_auc = 0
                    for kk in range(max_epochs):
                        if Auc[row][kk]>tmp_auc:
                            tmp_th = th_All[row][kk]
                            tmp_auc = Auc[row][kk]
                    acc, mcc, auc=Model.independTest(testData,testLabel,device,model_dir,batch_size,tmp_th)
                    Acc[row][-1] = acc
                    Mcc[row][-1] = mcc
                    Auc[row][-1] = auc
                    th_All[row][-1] = tmp_th
            if network_name[networki] == "Self-Attention+BiLSTM+No":
                
                for i,[train_index, test_index] in enumerate(kf.split(trainData)):
                    row+=1
                    row_one = type_name[typei]+"_"+cell_name[celli]+"_"+feature_name[featurei]+"_"+network_name[networki]+"_KFold_" + str(i)
                    row_name.append(row_one)
                    X_train = trainData[train_index]
                    X_test = trainData[test_index]
                    Y_train = trainLabel[train_index]
                    Y_test = trainLabel[test_index]
                    best_auc = 0
                    patience = 0
                    model_dir = "Model_Feature_Extraction/"+type_name[typei]+"_"+cell_name[celli]+"/"+\
                            feature_name[featurei]+"_"+network_name[networki]+"/KFold_" + str(i) + "/"
                    model = Model.ModelSB_Pro(trainData.shape[1],trainData.shape[2],False)
                    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")  #选择设备
                    os.environ['CUDA_VISIBLE_DEVICES'] = '0'
                    for j in range(max_epochs):
                        runningLoss,th = Model.train(model,X_train,Y_train,i,device,model_dir,batch_size)
                        acc, mcc, auc=Model.test(X_test,Y_test,best_auc,device,model_dir,batch_size,th)
                        Loss[row][j] = runningLoss
                        th_All[row][j] = th
                        Acc[row][j] = acc
                        Mcc[row][j] = mcc
                        Auc[row][j] = auc
                        if auc > best_auc:
                            best_auc=auc
                            patience=0
                        else:
                            patience+=1
                        if patience==max_patience:
                            break
                    tmp_th = 0.5
                    tmp_auc = 0
                    for kk in range(max_epochs):
                        if Auc[row][kk]>tmp_auc:
                            tmp_th = th_All[row][kk]
                            tmp_auc = Auc[row][kk]
                    acc, mcc, auc=Model.independTest(testData,testLabel,device,model_dir,batch_size,tmp_th)
                    Acc[row][-1] = acc
                    Mcc[row][-1] = mcc
                    Auc[row][-1] = auc
                    th_All[row][-1] = tmp_th
            if network_name[networki] == "BiLSTM+Self-Attention+No":
                
                for i,[train_index, test_index] in enumerate(kf.split(trainData)):
                    row+=1
                    row_one = type_name[typei]+"_"+cell_name[celli]+"_"+feature_name[featurei]+"_"+network_name[networki]+"_KFold_" + str(i)
                    row_name.append(row_one)
                    X_train = trainData[train_index]
                    X_test = trainData[test_index]
                    Y_train = trainLabel[train_index]
                    Y_test = trainLabel[test_index]
                    best_auc = 0
                    patience = 0
                    model_dir = "Model_Feature_Extraction/"+type_name[typei]+"_"+cell_name[celli]+"/"+\
                            feature_name[featurei]+"_"+network_name[networki]+"/KFold_" + str(i) + "/"
                    model = Model.ModelSB_Pro(trainData.shape[1],trainData.shape[2],False)
                    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")  #选择设备
                    os.environ['CUDA_VISIBLE_DEVICES'] = '0'
                    for j in range(max_epochs):
                        runningLoss,th = Model.train(model,X_train,Y_train,i,device,model_dir,batch_size)
                        acc, mcc, auc=Model.test(X_test,Y_test,best_auc,device,model_dir,batch_size,th)
                        Loss[row][j] = runningLoss
                        th_All[row][j] = th
                        Acc[row][j] = acc
                        Mcc[row][j] = mcc
                        Auc[row][j] = auc
                        if auc > best_auc:
                            best_auc=auc
                            patience=0
                        else:
                            patience+=1
                        if patience==max_patience:
                            break
                    tmp_th = 0.5
                    tmp_auc = 0
                    for kk in range(max_epochs):
                        if Auc[row][kk]>tmp_auc:
                            tmp_th = th_All[row][kk]
                            tmp_auc = Auc[row][kk]
                    acc, mcc, auc=Model.independTest(testData,testLabel,device,model_dir,batch_size,tmp_th)
                    Acc[row][-1] = acc
                    Mcc[row][-1] = mcc
                    Auc[row][-1] = auc
                    th_All[row][-1] = tmp_th
            if network_name[networki] == "Self-Attention+No":
                
                for i,[train_index, test_index] in enumerate(kf.split(trainData)):
                    row+=1
                    row_one = type_name[typei]+"_"+cell_name[celli]+"_"+feature_name[featurei]+"_"+network_name[networki]+"_KFold_" + str(i)
                    row_name.append(row_one)
                    X_train = trainData[train_index]
                    X_test = trainData[test_index]
                    Y_train = trainLabel[train_index]
                    Y_test = trainLabel[test_index]
                    best_auc = 0
                    patience = 0
                    model_dir = "Model_Feature_Extraction/"+type_name[typei]+"_"+cell_name[celli]+"/"+\
                            feature_name[featurei]+"_"+network_name[networki]+"/KFold_" + str(i) + "/"
                    model = Model.ModelS_Pro(trainData.shape[1],trainData.shape[2],False)
                    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")  #选择设备
                    os.environ['CUDA_VISIBLE_DEVICES'] = '0'
                    for j in range(max_epochs):
                        runningLoss,th = Model.train(model,X_train,Y_train,i,device,model_dir,batch_size)
                        acc, mcc, auc=Model.test(X_test,Y_test,best_auc,device,model_dir,batch_size,th)
                        Loss[row][j] = runningLoss
                        th_All[row][j] = th
                        Acc[row][j] = acc
                        Mcc[row][j] = mcc
                        Auc[row][j] = auc
                        if auc > best_auc:
                            best_auc=auc
                            patience=0
                        else:
                            patience+=1
                        if patience==max_patience:
                            break
                    tmp_th = 0.5
                    tmp_auc = 0
                    for kk in range(max_epochs):
                        if Auc[row][kk]>tmp_auc:
                            tmp_th = th_All[row][kk]
                            tmp_auc = Auc[row][kk]
                    acc, mcc, auc=Model.independTest(testData,testLabel,device,model_dir,batch_size,tmp_th)
                    Acc[row][-1] = acc
                    Mcc[row][-1] = mcc
                    Auc[row][-1] = auc
                    th_All[row][-1] = tmp_th

/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass shuffle=True, random_state=10 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)
/data/xiayunpeng/PycharmWorkspace/Whistle/LAModel.py:56: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  emb = torch.tensor(emb) + self.pe[:,:emb.shape[1]]


12.606578886508942


/data/xiayunpeng/PycharmWorkspace/Whistle/LAModel.py:56: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  emb = torch.tensor(emb) + self.pe[:,:emb.shape[1]]
/data/xiayunpeng/PycharmWorkspace/Whistle/LAModel.py:56: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  emb = torch.tensor(emb) + self.pe[:,:emb.shape[1]]


Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 0.208
auc on test set: 0.635
12.107134103775024


/data/xiayunpeng/PycharmWorkspace/Whistle/LAModel.py:56: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  emb = torch.tensor(emb) + self.pe[:,:emb.shape[1]]
/data/xiayunpeng/PycharmWorkspace/Whistle/LAModel.py:56: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  emb = torch.tensor(emb) + self.pe[:,:emb.shape[1]]


...

10.076518833637238
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 0.469
auc on test set: 0.802
10.05154824256897
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 0.466
auc on test set: 0.802
0.7110357330838061 0.42857716901337795 0.7829977950192943
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 0.429
auc on test set: 0.783
12.444197297096252
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 0.163
auc on test set: 0.597
12.30532294511795
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 0.150
auc on test set: 0.603
12.216941118240356
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 0.163
auc on test set: 0.608
12.137225270271301
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
S

12.350504398345947
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 0.205
auc on test set: 0.643
12.109263062477112
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 0.275
auc on test set: 0.697
11.703014194965363
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 0.355
auc on test set: 0.741
11.388719737529755
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 0.376
auc on test set: 0.750
11.085449278354645
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 0.423
auc on test set: 0.774
10.844575107097626
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 0.446
auc on test set: 0.784
10.744827210903168
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on tes

11.15380597114563
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 0.386
auc on test set: 0.758
11.07243686914444
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 0.397
auc on test set: 0.763
10.932027757167816
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 0.404
auc on test set: 0.764
10.793811798095703
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 0.422
auc on test set: 0.772
10.64473032951355
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 0.420
auc on test set: 0.771
10.527665078639984
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 0.430
auc on test set: 0.779
10.452066242694855
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test s

10.319650828838348
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 0.457
auc on test set: 0.789
10.232243716716766
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 0.449
auc on test set: 0.788
10.344344317913055
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 0.454
auc on test set: 0.791
10.245026528835297
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 0.440
auc on test set: 0.784
10.208029985427856
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 0.457
auc on test set: 0.791
10.184824883937836
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 0.453
auc on test set: 0.791
10.196696698665619
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on tes

10.165107548236847
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 0.448
auc on test set: 0.791
10.184322595596313
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 0.451
auc on test set: 0.792
10.143615067005157
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 0.455
auc on test set: 0.793
10.107940077781677
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 0.453
auc on test set: 0.790
10.102181375026703
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 0.451
auc on test set: 0.791
10.087031900882721
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 0.453
auc on test set: 0.792
10.102870643138885
Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on tes

In [6]:
network_name = ["Self-Attention+BiLSTM","BiLSTM+Self-Attention","BiLSTM",
                "Self-Attention","Self-Attention+BiLSTM+No","BiLSTM+Self-Attention+No","Self-Attention+No"]
#binary的
#第一次
# 0.794915483104568
# 0.7988531164383198
# 0.8035042065605149
# 0.7877696444515255
# 0.7848053505631951
# 0.7848571133970147
# 0.780838624192431
#第二次
#0.7929557860364784
#0.7958613479779745
#第三次
# 0.7957574758384329
# 0.7967739471024142
# 0.799691381435497
# 0.7862083931494728
#PCP的
# 0.7300439392383229
# 0.7887075801461421
# 0.7879413433398341
# 0.7607396748953242
# 0.5583581361441059
# 0.6702411123963026
# 0.5684933716521908
#NCPA的
# 0.7972365342917007
# 0.7990551307630279
# 0.8025121299282905
# 0.7907129297612425
# 0.7789443203611779
# 0.779358849195594
# 0.7805733313749862

In [14]:
for i in range(7):
    print(np.mean(Auc[5*i:5*i+5,-1]))

0.7972365342917007
0.7990551307630279
0.8025121299282905
0.7907129297612425
0.7789443203611779
0.779358849195594
0.7805733313749862


In [26]:
Auc[14]

array([0.77983743, 0.8033898 , 0.80377307, 0.80713079, 0.80765065,
       0.80779862, 0.81004202, 0.80787558, 0.80942766, 0.81236506,
       0.81271765, 0.81200867, 0.81313161, 0.81305977, 0.81320056,
       0.81217471, 0.81451098, 0.81491426, 0.81525002, 0.81589897,
       0.81719235, 0.81652986, 0.81790461, 0.81790164, 0.81762231,
       0.81705464, 0.8180805 , 0.81685423, 0.81765885, 0.81965269,
       0.81984171, 0.81962857, 0.82123319, 0.82035561, 0.82112154,
       0.82147085, 0.8215632 , 0.82146982, 0.82259204, 0.82226059,
       0.82348604, 0.8228808 , 0.82479645, 0.82298219, 0.8265083 ,
       0.82501441, 0.82510327, 0.82528552, 0.8270887 , 0.82781071,
       0.80532392])

In [107]:

# with open('some.csv', 'wb') as f:      # 采用二进制的方式处理可以省去很多问题
#     # 实例化csv.writer对象
#     writer = csv.writer(f)
#     # 用writerows方法将数据以指定形式保存
#     writer.writerows(np.append(np.array(row_name).reshape(90,1),Mcc,axis=1).encode())


# pd.DataFrame(np.append(np.array(row_name).reshape(90,1),Mcc,axis=1)).to_csv("out/Hek293_sysy_Full_Mcc.csv",index=0)
# pd.DataFrame(np.append(np.array(row_name).reshape(90,1),Auc,axis=1)).to_csv("out/Hek293_sysy_Full_Auc.csv",index=0)
# pd.DataFrame(np.append(np.array(row_name).reshape(90,1),Acc,axis=1)).to_csv("out/Hek293_sysy_Full_Acc.csv",index=0)
# pd.DataFrame(np.append(np.array(row_name).reshape(90,1),th_All,axis=1)).to_csv("out/Hek293_sysy_Full_th.csv",index=0)
# pd.DataFrame(np.append(np.array(row_name).reshape(90,1),Loss,axis=1)).to_csv("out/Hek293_sysy_Full_Loss.csv",index=0)
#pd.DataFrame(Mcc).to_csv("Mcc.csv",header=None,index=None)

In [40]:
#第一次binary_SB
print(np.max(np.array(pd.read_csv("out/Hek293_sysy_Full_Auc.csv",index_col=0).iloc[0:5])[:,0:50],axis=1),np.max(np.array(pd.read_csv("out/Hek293_sysy_Full_Auc.csv",index_col=0).iloc[0:5])[:,50:51],axis=1))
#第一次binary_SB
np.mean(np.max(np.array(pd.read_csv("out/Hek293_sysy_Full_Auc.csv",index_col=0).iloc[0:5])[:,0:50],axis=1)),np.mean(np.max(np.array(pd.read_csv("out/Hek293_sysy_Full_Auc.csv",index_col=0).iloc[0:5])[:,50:51],axis=1))

[0.8233499  0.8175945  0.79635115 0.81211521 0.79399129] [0.84373167 0.83517293 0.84334558 0.84246625 0.82929103]


(0.8086804121706352, 0.8388014929086915)

In [41]:
#第二次binary_SB，模型没变
print(np.max(Auc[:,0:200],axis=1),np.max(Auc[:,-1],axis=1))
np.mean(np.max(Auc[:,0:200],axis=1)),np.mean(np.max(Auc[:,200:201],axis=1))

[0.81447713 0.80373633 0.81573797 0.82020418 0.79345173] [0.8312803  0.83434103 0.84317789 0.83632681 0.83136597]


(0.8095214693436408, 0.8352983993953627)

In [8]:
#第三次binary_SB，残差加强
# print(np.max(Auc[:,0:50],axis=1),np.max(Auc[:,50:51],axis=1))
# np.mean(np.max(Auc[:,0:50],axis=1)),np.mean(np.max(Auc[:,50:51],axis=1))
#0.837,0.837

In [7]:
#第四次binary_SB，模型加深到6层，有强化残差,batch为512
print(np.max(Auc[:,0:50],axis=1),np.max(Auc[:,50:51],axis=1))
np.mean(np.max(Auc[:,0:50],axis=1)),np.mean(np.max(Auc[:,50:51],axis=1))

[0.80329221 0.81290254 0.79646772 0.80044289 0.79457094] [0.83647205 0.83705792 0.84017214 0.84174696 0.8368524 ]


(0.8015352602034493, 0.8384602943290618)

In [9]:
#第五次binary_SB，模型变浅到1层，有强化残差，batch为1024，epoch为200
print(np.max(Auc[:,0:200],axis=1),np.max(Auc[:,200:201],axis=1))
np.mean(np.max(Auc[:,0:200],axis=1)),np.mean(np.max(Auc[:,200:201],axis=1))

[0.82080131 0.8166629  0.81382799 0.8117616  0.80109246] [0.84365048 0.83581673 0.82816245 0.83284761 0.83688711]


(0.8128292520832832, 0.8354728787045367)

In [13]:
#第六次binary_SB，模型变深到6层，有强化残差，batch为1024，epoch为500
print(np.max(Auc[:,0:500],axis=1),np.max(Auc[:,500:501],axis=1))
np.mean(np.max(Auc[:,0:500],axis=1)),np.mean(np.max(Auc[:,500:501],axis=1))

[0.80497523 0.82755031 0.81827414 0.82601434 0.79496813] [0.84351184 0.84379574 0.84089236 0.84502425 0.83851282]


(0.8143564289063973, 0.8423474041395084)

In [6]:
#第七次binary_S，模型变深到6层，有强化残差，batch为1024，epoch为500
print(np.max(Auc[:,0:500],axis=1),np.max(Auc[:,500:501],axis=1))
np.mean(np.max(Auc[:,0:500],axis=1)),np.mean(np.max(Auc[:,500:501],axis=1))

[0.80808965 0.80334278 0.79937402 0.80502866 0.79180757] [0.84512266 0.84077244 0.84444273 0.83794425 0.83493441]


(0.8015285370414619, 0.8406432964485049)

In [6]:
#第八次CNN，模型变深到6层，有残差，batch为1024，epoch为500
print(np.max(Auc[:,0:500],axis=1),np.max(Auc[:,500:501],axis=1))
np.mean(np.max(Auc[:,0:500],axis=1)),np.mean(np.max(Auc[:,500:501],axis=1))

[0.7975377  0.79204675 0.78619736 0.79294545 0.78030662] [0.84005713 0.84264559 0.83873306 0.84226944 0.83855811]


(0.7898067772858911, 0.8404526673309665)

In [8]:
#第九次CNN，模型变深到6层，有残差，batch为1024，epoch为500，数据变为二维的164
print(np.max(Auc[:,0:500],axis=1),np.max(Auc[:,500:501],axis=1))
np.mean(np.max(Auc[:,0:500],axis=1)),np.mean(np.max(Auc[:,500:501],axis=1))

[0.79755212 0.79221436 0.78592195 0.79284585 0.78082608] [0.83585163 0.83978159 0.84076967 0.84289594 0.83713546]


(0.789872070054487, 0.8392868560288825)

In [7]:
#第十次SB，模型变深到12层，有残差，batch为1024，epoch为500
print(np.max(Auc[:,0:500],axis=1),np.max(Auc[:,500:501],axis=1))
np.mean(np.max(Auc[:,0:500],axis=1)),np.mean(np.max(Auc[:,500:501],axis=1))

[0.81809802 0.79971062 0.81985386 0.8233721  0.81344488] [0.84654043 0.84193236 0.85286805 0.84327922 0.83894392]


(0.8148958961581251, 0.8447127970611176)

In [6]:
#第十一次SB，模型变深到12层，有残差，batch为1024，epoch为500，学习率为0.01
print(np.max(Auc[:,0:500],axis=1),np.max(Auc[:,500:501],axis=1))
np.mean(np.max(Auc[:,0:500],axis=1)),np.mean(np.max(Auc[:,500:501],axis=1))

[0.79861883 0.79892701 0.79951305 0.81729311 0.78916233] [0.84213052 0.8337882  0.84194943 0.84605711 0.83037843]


(0.800702866324303, 0.8388607394292589)

In [6]:
#第十二次SB，模型变深到12层，有残差，batch为1024，epoch为500，学习率为0.01，数据量*5
print(np.max(Auc[:,0:500],axis=1),np.max(Auc[:,500:501],axis=1))
np.mean(np.max(Auc[:,0:500],axis=1)),np.mean(np.max(Auc[:,500:501],axis=1))

[0.82435969 0.82856256 0.82219348 0.8084582  0.81321979] [0.8401404  0.84320429 0.85082842 0.84596824 0.84992853]


(0.8193587419848856, 0.8460139765989597)

In [ ]:
#第十三次SB，模型变深到6层，有残差，batch为1024，epoch为500，学习率为0.001，数据量*5,使用三种不同的Fn层，再做加法得到结果
print(np.max(Auc[:,0:500],axis=1),np.max(Auc[:,500:501],axis=1))
np.mean(np.max(Auc[:,0:500],axis=1)),np.mean(np.max(Auc[:,500:501],axis=1))


In [ ]:
#第十三次binary_SB，5倍数据量，学习率0.01，out加了很多个输出
print(np.max(Auc[:,0:200],axis=1),np.max(Auc[:,-1],axis=1))
np.mean(np.max(Auc[:,0:200],axis=1)),np.mean(np.max(Auc[:,200:201],axis=1))
#np.mean(Auc[:,-1])
#0.8451686018869369

In [ ]:
#第十四次binary_SB，5倍数据量，学习率0.01,lstm加了个Attention
print(np.max(Auc[:,0:200],axis=1),np.max(Auc[:,-1],axis=1))
np.mean(np.max(Auc[:,0:200],axis=1)),np.mean(np.max(Auc[:,200:201],axis=1))
#array([0.85534271, 0.84883536, 0.83914385, 0.84593141, 0.84452867])
#0.8467564006668612

In [ ]:
#第十五次binary_SB，5倍数据量，学习率0.01，纯LSTM+h_n
print(np.max(Auc[:,0:200],axis=1),np.max(Auc[:,-1],axis=1))
np.mean(np.max(Auc[:,0:200],axis=1)),np.mean(np.max(Auc[:,200:201],axis=1))
#0.8458584705246668

In [ ]:
#第十六次binary_SB，5倍数据量，学习率0.01，纯LSTM+最后一层
print(np.max(Auc[:,0:200],axis=1),np.max(Auc[:,-1],axis=1))
np.mean(np.max(Auc[:,0:200],axis=1)),np.mean(np.max(Auc[:,200:201],axis=1))
#[0.8427204 , 0.84780565, 0.84181368, 0.85174631, 0.84780566]
#0.8463783389140913

# 以下为实验部分

In [1]:
#这是第一重循环~
#获取8个文件的文件名
for Folderi in range(len(Folder_name)):
    trainPosSeq_file = Folder_name[Folderi][0]
    trainNegSeq_file = Folder_name[Folderi][1]
    testPosSeq_file = Folder_name[Folderi][2]
    testNegSeq_file = Folder_name[Folderi][3]
    trainPosGene_file = Folder_name[Folderi][4]
    trainNegGene_file = Folder_name[Folderi][5]
    testPosGene_file = Folder_name[Folderi][6]
    testNegGene_file = Folder_name[Folderi][7]


NameError: name 'Folder_name' is not defined

In [6]:
#获取文件信息
trainPos_seq = pd.read_csv(trainPosSeq_file,header=None)
trainNeg_seq = pd.read_csv(trainNegSeq_file,header=None)
testPos_seq = pd.read_csv(testPosSeq_file,header=None)
testNeg_seq = pd.read_csv(testNegSeq_file,header=None)
trainPos_gene = pd.read_csv(trainPosGene_file)
trainNeg_gene = pd.read_csv(trainNegGene_file)
testPos_gene = pd.read_csv(testPosGene_file)
testNeg_gene = pd.read_csv(testNegGene_file)

In [7]:
#序列转41-nt长度
trainPos_seq = long_short(trainPos_seq)
trainNeg_seq = long_short(trainNeg_seq)
testPos_seq = long_short(testPos_seq)
testNeg_seq = long_short(testNeg_seq)

In [8]:
#基因转35长度
trainPos_gene = trainPos_gene.iloc[:,[0,1,3,4,5,6,7,8,9,10,11,13,14,16,17,19,23,22,25,20,21,44,45,46,47,48,49,50,65,66,69,55,51,57,56]]
trainNeg_gene = trainNeg_gene.iloc[:,[0,1,3,4,5,6,7,8,9,10,11,13,14,16,17,19,23,22,25,20,21,44,45,46,47,48,49,50,65,66,69,55,51,57,56]]
testPos_gene = testPos_gene.iloc[:,[0,1,3,4,5,6,7,8,9,10,11,13,14,16,17,19,23,22,25,20,21,44,45,46,47,48,49,50,65,66,69,55,51,57,56]]
testNeg_gene = testNeg_gene.iloc[:,[0,1,3,4,5,6,7,8,9,10,11,13,14,16,17,19,23,22,25,20,21,44,45,46,47,48,49,50,65,66,69,55,51,57,56]]

In [9]:
trainPos_seq,trainPos_gene = check_N(trainPos_seq,np.array(trainPos_gene))
trainNeg_seq,trainNeg_gene = check_N(trainNeg_seq,np.array(trainNeg_gene))
testPos_seq,testPos_gene = check_N(testPos_seq,np.array(testPos_gene))
testNeg_seq,testNeg_gene = check_N(testNeg_seq,np.array(testNeg_gene))

In [10]:
#基因变Float
trainPos_gene = np.float32(trainPos_gene)
trainNeg_gene = np.float32(trainNeg_gene)
testPos_gene = np.float32(testPos_gene)
testNeg_gene = np.float32(testNeg_gene)

In [11]:
#打乱顺序，再取等长
trainNeg_seq, trainNeg_gene = shuffle(trainNeg_seq,trainNeg_gene, random_state=1)
testNeg_seq, testNeg_gene = shuffle(testNeg_seq, testNeg_gene, random_state=1)

trainNeg_seq = trainNeg_seq[:len(trainPos_seq)]
trainNeg_gene = trainNeg_gene[:len(trainPos_gene)]
testNeg_seq = testNeg_seq[:len(testPos_seq)]
testNeg_gene = testNeg_gene[:len(testPos_gene)]

# print(trainPos_seq.shape,trainNeg_seq.shape,trainPos_gene.shape,trainNeg_gene.shape,testPos_seq.shape,
#       testNeg_seq.shape,testPos_gene.shape,testNeg_gene.shape)

In [17]:
#第二重循环
for featurei in range(len(feature_name)):
    if feature_name[featurei] == "binary":
        trainPos = ensembleFeature.Binary(trainPos_seq)
        trainNeg = ensembleFeature.Binary(trainNeg_seq)
        testPos = ensembleFeature.Binary(testPos_seq)
        testNeg = ensembleFeature.Binary(testNeg_seq)
    elif feature_name[featurei] == "NCPA":
        trainPos = ensembleFeature.NCPA(trainPos_seq)
        trainNeg = ensembleFeature.NCPA(trainNeg_seq)
        testPos = ensembleFeature.NCPA(testPos_seq)
        testNeg = ensembleFeature.NCPA(testNeg_seq)
    elif feature_name[featurei] == "gene":
        trainPos = trainPos_gene
        trainNeg = trainNeg_gene
        testPos = testPos_gene
        testNeg = testNeg_gene
    elif feature_name[featurei] == "emb":
        trainPos = ensembleFeature.emb_seqs(trainPos_seq)
        trainNeg = ensembleFeature.emb_seqs(trainNeg_seq)
        testPos = ensembleFeature.emb_seqs(testPos_seq)
        testNeg = ensembleFeature.emb_seqs(testNeg_seq)
    elif feature_name[featurei] == "PSNP":
        trainPos,trainNeg,testPos,testNeg = ensembleFeature.PSNP(trainPos_seq,trainNeg_seq,testPos_seq,testNeg_seq)
    elif feature_name[featurei] == "ENAC":
        trainPos = ensembleFeature.ENAC(trainPos_seq)
        trainNeg = ensembleFeature.ENAC(trainNeg_seq)
        testPos = ensembleFeature.ENAC(testPos_seq)
        testNeg = ensembleFeature.ENAC(testNeg_seq)
    if len(testNeg.shape)==2:
        trainPos = trainPos.reshape(trainPos.shape[0],1,trainPos.shape[1])
        trainNeg = trainNeg.reshape(trainNeg.shape[0],1,trainNeg.shape[1])
        testPos = testPos.reshape(testPos.shape[0],1,testPos.shape[1])
        testNeg = testNeg.reshape(testNeg.shape[0],1,testNeg.shape[1])

{'A': 0, 'C': 1, 'G': 2, 'T': 3}


In [ ]:
trainPos_label = np.ones(len(trainPos))
trainNeg_label = np.zeros(len(trainNeg))
testPos_label = np.ones(len(testPos))
testNeg_label = np.zeros(len(testNeg))

trainData = np.append(trainPos,trainNeg,axis=0)
testData = np.append(testPos,testNeg,axis = 0)
trainLabel = np.append(trainPos_label,trainNeg_label,axis = 0)
testLabel = np.append(testPos_label,testNeg_label,axis = 0)

kf = KFold(5,True,10)
#第三重循环
for networki in range(len(network_name)):
#     if network_name[networki] == "Self-Attention+BiLSTM":
#         trainLoss = []
#         Acc = []
#         Mcc = []
#         Auc = []
#         max_epochs = 50
#         patience = 10
#         batch_size = 128
          
#         for i,[train_index, test_index] in enumerate(kf.split(trainData)):
#             kf_trainLoss = []
#             kf_Acc = []
#             kf_Mcc = []
#             kf_Auc = []

#             X_train = trainData[train_index]
#             X_test = trainData[test_index]
#             Y_train = trainLabel[train_index]
#             Y_test = trainLabel[test_index]
#             best_auc = 0
#             patience = 0
#             model_dir = "Model_Feature_Extraction/"+type_name[typei]+"_"+cell_name[celli]+"/"+\
#                     feature_name[featurei]+"_"+network_name[networki]+"/KFold_" + str(i) + "/"
#             model = Model.ModelSB_Pro(trainData.shape[1],trainData.shape[2])
#             device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")  #选择设备
#             os.environ['CUDA_VISIBLE_DEVICES'] = '0'
#             for j in range(max_epochs):
#                 runningLoss,th = Model.train(model,X_train,Y_train,i,device,model_dir,batch_size)
#                 acc, mcc, auc=Model.test(X_test,Y_test,best_auc,device,model_dir,batch_size,th)
#                 kf_trainLoss.append(runningLoss)
#                 kf_Acc.append(acc)
#                 kf_Mcc.append(mcc)
#                 kf_Auc.append(auc)
#                 if auc > best_auc:
#                     best_auc=auc
#                     patience=0
#                 else:
#                     patience+=1
#                 if patience==10:
#                     break
#             trainLoss.append(kf_trainLoss)
#             Acc.append(kf_Acc)
#             Mcc.append(kf_Mcc)
#             Auc.append(kf_Auc)
    if network_name[networki] == "BiLSTM+Self-Attention":
        trainLoss = []
        Acc = []
        Mcc = []
        Auc = []
        max_epochs = 50
        patience = 10
        batch_size = 128
          
        for i,[train_index, test_index] in enumerate(kf.split(trainData)):
            kf_trainLoss = []
            kf_Acc = []
            kf_Mcc = []
            kf_Auc = []

            X_train = trainData[train_index]
            X_test = trainData[test_index]
            Y_train = trainLabel[train_index]
            Y_test = trainLabel[test_index]
            best_auc = 0
            patience = 0
            model_dir = "Model_Feature_Extraction/"+type_name[typei]+"_"+cell_name[celli]+"/"+\
                    feature_name[featurei]+"_"+network_name[networki]+"/KFold_" + str(i) + "/"
            model = Model.ModelBS_Pro(trainData.shape[1],trainData.shape[2])
            device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")  #选择设备
            os.environ['CUDA_VISIBLE_DEVICES'] = '0'
            for j in range(max_epochs):
                runningLoss,th = Model.train(model,X_train,Y_train,i,device,model_dir,batch_size)
                acc, mcc, auc=Model.test(X_test,Y_test,best_auc,device,model_dir,batch_size,th)
                kf_trainLoss.append(runningLoss)
                kf_Acc.append(acc)
                kf_Mcc.append(mcc)
                kf_Auc.append(auc)
                if auc > best_auc:
                    best_auc=auc
                    patience=0
                else:
                    patience+=1
                if patience==10:
                    break
            trainLoss.append(kf_trainLoss)
            Acc.append(kf_Acc)
            Mcc.append(kf_Mcc)
            Auc.append(kf_Auc)

#     if network_name[networki] == "BILSTM":
#         trainLoss = []
#         Acc = []
#         Mcc = []
#         Auc = []
#         max_epochs = 50
#         patience = 10
#         batch_size = 128
          
#         for i,[train_index, test_index] in enumerate(kf.split(trainData)):
#             kf_trainLoss = []
#             kf_Acc = []
#             kf_Mcc = []
#             kf_Auc = []

#             X_train = trainData[train_index]
#             X_test = trainData[test_index]
#             Y_train = trainLabel[train_index]
#             Y_test = trainLabel[test_index]
#             best_auc = 0
#             patience = 0
#             model_dir = "Model_Feature_Extraction/"+type_name[typei]+"_"+cell_name[celli]+"/"+\
#                     feature_name[featurei]+"_"+network_name[networki]+"/KFold_" + str(i) + "/"
#             model = Model.ModelB_Pro(trainData.shape[1],trainData.shape[2])
#             device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")  #选择设备
#             #model = model.to(device)
#             os.environ['CUDA_VISIBLE_DEVICES'] = '0'
#             #os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
#             for j in range(max_epochs):
#                 runningLoss,th = Model.train(model,X_train,Y_train,i,device,model_dir,batch_size)
#                 acc, mcc, auc=Model.test(X_test,Y_test,best_auc,device,model_dir,batch_size,th)
#                 kf_trainLoss.append(runningLoss)
#                 kf_Acc.append(acc)
#                 kf_Mcc.append(mcc)
#                 kf_Auc.append(auc)
#                 if auc > best_auc:
#                     best_auc=auc
#                     patience=0
#                 else:
#                     patience+=1
#                 if patience==10:
#                     break
#             trainLoss.append(kf_trainLoss)
#             Acc.append(kf_Acc)
#             Mcc.append(kf_Mcc)
#             Auc.append(kf_Auc)
            

/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass shuffle=True, random_state=10 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)
/data/xiayunpeng/PycharmWorkspace/Whistle/Model.py:56: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  emb = torch.tensor(emb) + self.pe[:,:emb.shape[1]]


113.93126213550568


/data/xiayunpeng/PycharmWorkspace/Whistle/Model.py:56: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  emb = torch.tensor(emb) + self.pe[:,:emb.shape[1]]


Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 0.428
auc on test set: 0.773


/data/xiayunpeng/PycharmWorkspace/Whistle/Model.py:56: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  emb = torch.tensor(emb) + self.pe[:,:emb.shape[1]]


...

/data/xiayunpeng/PycharmWorkspace/Whistle/Model.py:56: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  emb = torch.tensor(emb) + self.pe[:,:emb.shape[1]]


108.9126917719841


/data/xiayunpeng/PycharmWorkspace/Whistle/Model.py:56: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  emb = torch.tensor(emb) + self.pe[:,:emb.shape[1]]


Accuracy on test set: 0 %
Sensitivity on test set: 0 %
Speciality on test set: 0 %
MCC on test set: 0.398
auc on test set: 0.762


/data/xiayunpeng/PycharmWorkspace/Whistle/Model.py:56: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  emb = torch.tensor(emb) + self.pe[:,:emb.shape[1]]


In [ ]:
for networki in range(len(network_name)):
    if network_name[networki] == "BILSTM+Self-Attention":
        Acc = []
        Mcc = []
        Auc = []
        patience = 10
        batch_size = 128
        for i in range(5):
            model_dir = "Model_Feature_Extraction/"+type_name[typei]+"_"+cell_name[celli]+"/"+\
                    feature_name[featurei]+"_"+network_name[networki]+"/KFold_" + str(i) + "/"

            model = Model.ModelBS_Pro(trainData.shape[1],trainData.shape[2])
            device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")  #选择设备
            os.environ['CUDA_VISIBLE_DEVICES'] = '0'
            
            acc, mcc, auc=Model.independTest(testData,testLabel,device,model_dir,batch_size,0.5)
            print(acc,mcc,auc)
            Acc.append(acc)
            Mcc.append(mcc)
            Auc.append(auc)

In [99]:
Mcc

[0.9444697447947306,
 0.9562921940953324,
 0.9820151174843108,
 0.9715128806216057,
 0.9006478020200869]